<center> </center>

<center><font size=5 face="Helvetica" color=#76B900><b>
CUDA MODE: Lecture 1
</b></font></center>

<center><font face="Helvetica" size=3><b>Ang Chen</b></font></center>
<center><font face="Helvetica" size=3>September, 2024</font></center>

***

# Pytorch square

In [1]:
import torch

In [2]:
a = torch.tensor([1., 2., 3.])

print(torch.square(a))
print(a ** 2)
print(a * a)

tensor([1., 4., 9.])
tensor([1., 4., 9.])
tensor([1., 4., 9.])


In [3]:
def time_pytorch_function(func, input):
    # CUDA IS ASYNC so can't use python time module
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)

    # Warmup
    for _ in range(5):
        func(input)

    start.record()
    func(input)
    end.record()
    torch.cuda.synchronize()
    return start.elapsed_time(end)

In [4]:
b = torch.randn(10000, 10000).cuda()

def square_2(a):
    return a * a

def square_3(a):
    return a ** 2

In [5]:
time_pytorch_function(torch.square, b)
time_pytorch_function(square_2, b)
time_pytorch_function(square_3, b)

print("=============")
print("Profiling torch.square")
print("=============")

Profiling torch.square


In [6]:
with torch.autograd.profiler.profile(use_cuda=True) as prof:
    torch.square(b)

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

print("=============")
print("Profiling a * a")
print("=============")

---------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                 Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
---------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
         aten::square        22.83%     189.000us       100.00%     828.000us     828.000us     128.000us         0.12%     107.373ms     107.373ms             1  
            aten::pow        76.93%     637.000us        77.17%     639.000us     639.000us     107.240ms        99.88%     107.245ms     107.245ms             1  
    aten::result_type         0.12%       1.000us         0.12%       1.000us       1.000us       3.000us         0.00%       3.000us       3.000us             1  
             at

In [7]:
with torch.autograd.profiler.profile(use_cuda=True) as prof:
    square_2(b)

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

print("=============")
print("Profiling a ** 2")
print("=============")

-------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
         Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
    aten::mul       100.00%      52.000us       100.00%      52.000us      52.000us     121.031ms       100.00%     121.031ms     121.031ms             1  
-------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
Self CPU time total: 52.000us
Self CUDA time total: 121.031ms

Profiling a ** 2


In [8]:
with torch.autograd.profiler.profile(use_cuda=True) as prof:
    square_3(b)

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

---------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                 Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
---------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
            aten::pow        97.01%      65.000us       100.00%      67.000us      67.000us     121.912ms       100.00%     121.915ms     121.915ms             1  
             aten::to         1.49%       1.000us         1.49%       1.000us       1.000us       2.000us         0.00%       2.000us       2.000us             1  
    aten::result_type         1.49%       1.000us         1.49%       1.000us       1.000us       1.000us         0.00%       1.000us       1.000us             1  
---------------